In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget "https://data.jakarta.go.id/dataset/e9436d31-d6e6-48d6-9aed-686712082146/resource/50231547-1203-42f7-8fa6-fffc3bd4e8ef/download/Data-Penjualan-dan-Pajak-BBM-Tahun-2016.csv"

--2023-02-04 03:50:33--  https://data.jakarta.go.id/dataset/e9436d31-d6e6-48d6-9aed-686712082146/resource/50231547-1203-42f7-8fa6-fffc3bd4e8ef/download/Data-Penjualan-dan-Pajak-BBM-Tahun-2016.csv
Resolving data.jakarta.go.id (data.jakarta.go.id)... 103.209.7.64
Connecting to data.jakarta.go.id (data.jakarta.go.id)|103.209.7.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/x-comma-separated-values]
Saving to: ‘Data-Penjualan-dan-Pajak-BBM-Tahun-2016.csv’

Data-Penjualan-dan-     [ <=>                ]   7.12K  --.-KB/s    in 0s      

2023-02-04 03:50:34 (65.1 MB/s) - ‘Data-Penjualan-dan-Pajak-BBM-Tahun-2016.csv’ saved [7289]



In [65]:
data_bbm=pd.read_csv("Data-Penjualan-dan-Pajak-BBM-Tahun-2016.csv")

# EDA

In [66]:
data_bbm.head()

,bulan,stasiun_pengisian_bahanbakar,jenis_bahanbakar,jumlah_penjualan,penerimaan_pajak
0,1,Pertamina,Bio Solar,42407041,10525855948
1,1,Pertamina,Solar,23003789,6199987321
2,1,Pertamina,Premium,134009663,41428120949
3,1,Pertamina,Pertalite,15232000,5229147971
4,1,Pertamina,Pertamax,34835000,12816101073


In [67]:
data_bbm.columns

Index(['bulan', 'stasiun_pengisian_bahanbakar', 'jenis_bahanbakar',
       'jumlah_penjualan', 'penerimaan_pajak'],
      dtype='object')

In [68]:
# defining a function for getting percentages
def crosstab_function(df, var):
    tab = pd.DataFrame(pd.crosstab(df[var], df["stasiun_pengisian_bahanbakar"], margins=True)).reset_index()
    # tab['Percentage'] = tab[1] / tab['All'] * 100
    tab.columns = [var, 'Pertamina', 'Shell',"Total", "ALL"]
    return tab

In [69]:
crosstab_function(data_bbm,"stasiun_pengisian_bahanbakar")

,stasiun_pengisian_bahanbakar,Pertamina,Shell,Total,ALL
0,Pertamina,132,0,0,132
1,Shell,0,36,0,36
2,Total,0,0,36,36
3,All,132,36,36,204


In [70]:
for col in data_bbm.select_dtypes(include='object'):
    print(f"{col:30} ----> {data_bbm[col].nunique()} unique values")

stasiun_pengisian_bahanbakar   ----> 3 unique values
jenis_bahanbakar               ----> 16 unique values


## Check data if null

In [71]:
data_bbm.isnull().sum()

bulan                           0
stasiun_pengisian_bahanbakar    0
jenis_bahanbakar                0
jumlah_penjualan                0
penerimaan_pajak                0
dtype: int64

In [72]:
data_bbm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   bulan                         204 non-null    int64 
 1   stasiun_pengisian_bahanbakar  204 non-null    object
 2   jenis_bahanbakar              204 non-null    object
 3   jumlah_penjualan              204 non-null    int64 
 4   penerimaan_pajak              204 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 8.1+ KB


In [73]:
data_bbm["bulan"]=data_bbm["bulan"].astype("object")

The data not null 

# Data Preprocessing

In [74]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [75]:
data_bbm.head()

,bulan,stasiun_pengisian_bahanbakar,jenis_bahanbakar,jumlah_penjualan,penerimaan_pajak
0,1,Pertamina,Bio Solar,42407041,10525855948
1,1,Pertamina,Solar,23003789,6199987321
2,1,Pertamina,Premium,134009663,41428120949
3,1,Pertamina,Pertalite,15232000,5229147971
4,1,Pertamina,Pertamax,34835000,12816101073


## Doing hot encoder use 
`pandas.get_dummies`

In [76]:
fitur=pd.get_dummies(data_bbm[["stasiun_pengisian_bahanbakar","jenis_bahanbakar"]])

In [77]:
fitur

,stasiun_pengisian_bahanbakar_Pertamina,stasiun_pengisian_bahanbakar_Shell,stasiun_pengisian_bahanbakar_Total,jenis_bahanbakar_BBG,jenis_bahanbakar_Bio Solar,jenis_bahanbakar_Dexlite,jenis_bahanbakar_Diesel,jenis_bahanbakar_Performance 92,jenis_bahanbakar_Performance 95,jenis_bahanbakar_Pertalite,jenis_bahanbakar_Pertamax,jenis_bahanbakar_Pertamax Plus,jenis_bahanbakar_Pertamax Racing,jenis_bahanbakar_Pertamax Turbo,jenis_bahanbakar_Pertamina Dex,jenis_bahanbakar_Premium,jenis_bahanbakar_Solar,jenis_bahanbakar_Super,jenis_bahanbakar_Vpower
0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
200,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
201,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
202,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [78]:
data_bbm=pd.concat([data_bbm,fitur],axis=1)

In [80]:
data_bbm.drop(columns=(["stasiun_pengisian_bahanbakar","jenis_bahanbakar"]),inplace=True)

In [81]:
data_bbm

,bulan,jumlah_penjualan,penerimaan_pajak,stasiun_pengisian_bahanbakar_Pertamina,stasiun_pengisian_bahanbakar_Shell,stasiun_pengisian_bahanbakar_Total,jenis_bahanbakar_BBG,jenis_bahanbakar_Bio Solar,jenis_bahanbakar_Dexlite,jenis_bahanbakar_Diesel,...,jenis_bahanbakar_Pertalite,jenis_bahanbakar_Pertamax,jenis_bahanbakar_Pertamax Plus,jenis_bahanbakar_Pertamax Racing,jenis_bahanbakar_Pertamax Turbo,jenis_bahanbakar_Pertamina Dex,jenis_bahanbakar_Premium,jenis_bahanbakar_Solar,jenis_bahanbakar_Super,jenis_bahanbakar_Vpower
0,1,42407041,10525855948,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,23003789,6199987321,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,134009663,41428120949,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,15232000,5229147971,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,34835000,12816101073,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,12,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200,12,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
201,12,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
202,12,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Modelling

In [87]:
x=data_bbm.drop(columns=(["jumlah_penjualan","penerimaan_pajak"]))
y=data_bbm.jumlah_penjualan

In [86]:
from sklearn.linear_model import LinearRegression

In [89]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
model = make_pipeline(StandardScaler(), LinearRegression())
cv_result = cross_validate(model, x, y, cv=5,return_train_score=True)
cv_result

{'fit_time': array([0.01821136, 0.00506401, 0.00489211, 0.00506496, 0.00484157]),
 'score_time': array([0.00322556, 0.00218606, 0.00200963, 0.00199175, 0.00231242]),
 'test_score': array([0.7796799 , 0.86250219, 0.86266361, 0.18196012, 0.        ]),
 'train_score': array([0.65161047, 0.6685367 , 0.65586247, 0.78151085, 0.86091845])}

In [91]:
model_linear_reg=pd.DataFrame(cv_result)

In [92]:
model_linear_reg

,fit_time,score_time,test_score,train_score
0,0.018211,0.003226,0.779680,0.651610
1,0.005064,0.002186,0.862502,0.668537
2,0.004892,0.002010,0.862664,0.655862
3,0.005065,0.001992,0.181960,0.781511
4,0.004842,0.002312,0.000000,0.860918


In [94]:
print(f'Test score menggunakan linear regression: {cv_result["test_score"].mean()}')
print(f'Ttrain score menggunakan linear regression: {cv_result["train_score"].mean()}')

Test score menggunakan linear regression: 0.5373611632859647
Ttrain score menggunakan linear regression: 0.7236877882553726


In [96]:
from sklearn.tree import DecisionTreeRegressor

In [99]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
model_decTreeRegressor = make_pipeline(StandardScaler(), DecisionTreeRegressor())
cv_result = cross_validate(model_decTreeRegressor, x, y, cv=5,return_train_score=True)
cv_result

{'fit_time': array([0.01456285, 0.00647926, 0.00470495, 0.00757647, 0.00483894]),
 'score_time': array([0.00333357, 0.00209117, 0.00254869, 0.00215364, 0.00194836]),
 'test_score': array([0.98943257, 0.99533317, 0.96758294, 0.80655194, 0.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [100]:
model_tree_dec_reg=pd.DataFrame(cv_result)

In [101]:
model_tree_dec_reg

,fit_time,score_time,test_score,train_score
0,0.014563,0.003334,0.989433,1.0
1,0.006479,0.002091,0.995333,1.0
2,0.004705,0.002549,0.967583,1.0
3,0.007576,0.002154,0.806552,1.0
4,0.004839,0.001948,0.000000,1.0


In [104]:
print(f'{model_tree_dec_reg["test_score"].mean()}')
print(f'{model_tree_dec_reg["train_score"].mean()}')

0.7517801236885042
1.0
